#                                   Industry- Manufacturing

###  Importing Modules

In [273]:
from gamspy import Container, Set, Variable, Parameter, Model, Product, Sum, Sense, Equation
import numpy as np
import pandas as pd

### Data

In [274]:
m = Container ()
# Set
Products = Set(m, "Products", records = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6",
                                        "Product 7", "Product 8"])
Sites = Set(m, "Sites", records = ["Sites 1", "Sites 2", "Sites 3", "Sites 4", "Sites 5"])
# Parameter

A = Parameter(
    container = m,
    name = "A",
    domain = [Products, Sites],
    records = np.array([[1, 7, 1, 0, 6], [2, 3, 5, 2, 1], [4, 1, 4, 5, 1], [1, 5, 3, 1, 5], [2, 1, 2, 1, 0], 
                       [1, 4, 5, 5, 4], [3, 6, 2, 1, 2], [6, 2, 5, 1, 8]])
)

b = Parameter(
    container = m,
    name = "b",
    domain = Sites,
    records = np.array([2, 4, 3, 2, 2])
)

s = Parameter(
    container = m,
    name = "s",
    domain = Sites,
    records = np.array([1, 1, 1, 2, 1])
)

l = Parameter(
    container = m,
    name = "l",
    domain = Products,
    records = np.array([20, 10, 30, 40, 50, 20, 20, 10])
)

q = Parameter(
    container = m,
    name = "q",
    domain = Products,
    records = np.array([50, 20, 115, 65, 115, 65, 30, 60])
)

d1 = Parameter(
    container = m,
    name = "d1",
    domain = Products,
    records = np.array([9, 8, 6, 4, 10, 4, 3, 2])
)

d2 = Parameter(
    container = m,
    name = "d2",
    domain = Products,
    records = np.array([5, 4, 7, 5, 6, 3, 8, 7])
)

x = Variable(
    container = m,
    name = "x",
    domain = Sites,
    type = "Positive"
)

y1 = Variable(
    container = m,
    name = "y1",
    domain = Sites,
    type = "Positive"
)

y2 = Variable(
    container = m,
    name = "y2",
    domain = Sites,
    type = "Positive"
)

z1 = Variable(
    container = m,
    name = "z1",
    domain = Products,
    type = "Positive"
)

z2 = Variable(
    container = m,
    name = "z1",
    domain = Products,
    type = "Positive"
)

### Objective Function

In [275]:
objFuntion = Sum([Sites], b[Sites] * x[Sites]) + 0.5 * (Sum([Products], (l[Products] - q[Products]) * z1[Products]) - Sum([Sites], s[Sites] * y1[Sites])) + 0.5 *(Sum([Products], (l[Products] - q[Products]) * z2[Products]) - Sum([Sites], s[Sites] * y2[Sites]))

### Subject to 

In [276]:
def1 = Equation(container = m, name = "def1", domain = [Products])
def1[Products] = z1[Products] >= 0

def2 = Equation(container = m, name = "def2", domain = [Products])
def2[Products] = z2[Products] >= 0

def3 = Equation(container = m, name = "def3", domain = [Products])
def3[Products] = z1[Products] <= d1[Products]

def4 = Equation(container = m, name = "def4", domain = [Products])
def4[Products] = z2[Products] <= d2[Products]

def5 = Equation(container = m, name = "def5", domain = [Sites])
def5[Sites] = y1[Sites] >= 0

def6 = Equation(container = m, name = "def6", domain = [Sites])
def6[Sites] = y2[Sites] >= 0

def7 = Equation(container = m, name = "def7", domain = [Sites])
def7[Sites] = y1[Sites] == (x[Sites] - Sum([Products], A[Products, Sites] * z1[Products]))

def8 = Equation(container = m, name = "def8", domain = [Sites])
def8[Sites] = y2[Sites] == (x[Sites] - Sum([Products] ,A[Products, Sites] * z2[Products]))

def9 = Equation(container = m, name = "def9", domain = [Sites])
def9[Sites] = x[Sites] >= 0


### Model

In [277]:
result = Model (
    container = m,
    name = "result",
    equations = m.getEquations(),
    problem = "LP",
    sense = Sense.MIN,
    objective = objFuntion
)

### Solve

In [278]:
result.solve()

In [279]:
print(result.objective_value)

-588.0


In [280]:
print(x.records)

     Sites  level  marginal  lower  upper  scale
0  Sites 1   36.0       0.0    0.0    inf    1.0
1  Sites 2   12.0       0.0    0.0    inf    1.0
2  Sites 3   36.0       0.0    0.0    inf    1.0
3  Sites 4   36.0       0.0    0.0    inf    1.0
4  Sites 5    6.0       0.0    0.0    inf    1.0


In [281]:
print(y1.records)

     Sites  level  marginal  lower  upper  scale
0  Sites 1    0.0       1.0    0.0    inf    1.0
1  Sites 2    0.0       3.0    0.0    inf    1.0
2  Sites 3    0.0       2.0    0.0    inf    1.0
3  Sites 4    0.0      -0.0    0.0    inf    1.0
4  Sites 5    0.0       1.0    0.0    inf    1.0


In [282]:
print(y2.records)

     Sites  level  marginal  lower  upper  scale
0  Sites 1    0.0       0.0    0.0    inf    1.0
1  Sites 2    0.0       0.0    0.0    inf    1.0
2  Sites 3    0.0       0.0    0.0    inf    1.0
3  Sites 4    0.0       0.0    0.0    inf    1.0
4  Sites 5    0.0       0.0    0.0    inf    1.0


In [283]:
print(z1.records)

    Products  level  marginal  lower  upper  scale
0  Product 1    0.0      15.0    0.0    inf    1.0
1  Product 2    0.0      27.0    0.0    inf    1.0
2  Product 3    6.0       0.0    0.0    inf    1.0
3  Product 4    0.0      18.0    0.0    inf    1.0
4  Product 5    6.0       0.0    0.0    inf    1.0
5  Product 6    0.0       6.0    0.0    inf    1.0
6  Product 7    0.0      32.0    0.0    inf    1.0
7  Product 8    0.0       3.0    0.0    inf    1.0


In [272]:
print(z2.records)

    Products  level  marginal  lower  upper  scale
0  Product 1    0.0      15.0    0.0    inf    1.0
1  Product 2    0.0      27.0    0.0    inf    1.0
2  Product 3    6.0       0.0    0.0    inf    1.0
3  Product 4    0.0      18.0    0.0    inf    1.0
4  Product 5    6.0       0.0    0.0    inf    1.0
5  Product 6    0.0       6.0    0.0    inf    1.0
6  Product 7    0.0      32.0    0.0    inf    1.0
7  Product 8    0.0       3.0    0.0    inf    1.0
